In [5]:
import numpy as np
import pandas as pd
import duckdb
import pygwalker as pg

%reload_ext sql
conn = duckdb.connect('X:\\nba_data\\odds_data\\odds.db')
%sql conn --alias duckdb

%config SqlMagic.displaylimit = 25

Tip: You may define configurations in C:\Users\cwood\Documents\bball_proj\wood-ball\pyproject.toml or C:\Users\cwood\.jupysql\config.

Did not find user configurations in C:\Users\cwood\Documents\bball_proj\wood-ball\pyproject.toml.

In [4]:
%%sql
show tables

Running query in 'duckdb'

name
nba_champ_odds
nba_game_odds
ncaa_champ_odds


In [6]:
df = conn.sql("""
select * from nba_game_odds
""").df()
pg.walk(df, kernel_computation=True)

Box(children=(HTML(value='\n<div id="ifr-pyg-000626d364929fb72hE6RALMkwPaNxIy" style="height: auto">\n    <hea…

In [ ]:
df = conn.sql("""
with first as (
    select
    name,
    bookmakers_title,
    avg(price) as avg_odds
    from nba_champ_odds
    group by name, bookmakers_title
)
pivot first
on bookmakers_title
using avg(avg_odds)
""").df()

In [ ]:
%%sql
with first as (
    select
    bookmakers_title, name,
    avg(price) as avg_odds,
    from nba_champ_odds
    group by name, bookmakers_title
    order by avg_odds asc
)
select 
RANK() OVER (ORDER BY avg_odds) AS rank,
*
from first

In [ ]:
from sklearn.cluster import KMeans

# Reshape the data to a 2D array for KMeans
X = df['avg_odds'].values.reshape(-1, 1)

# Choose the number of clusters (k)
k = 4

# Apply K-Means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(X)

# Assign the cluster labels to your original DataFrame
df['cluster'] = kmeans.labels_